DynaBench

https://github.com/bvidgen/Dynamically-Generated-Hate-Speech-Dataset

https://arxiv.org/pdf/2012.15761.pdf

In [10]:
# !wget https://raw.githubusercontent.com/bvidgen/Dynamically-Generated-Hate-Speech-Dataset/main/Dynamically%20Generated%20Hate%20Dataset%20v0.2.3.csv

In [1]:
import pandas as pd

In [2]:
df_dynabench = pd.read_csv("raw_data/Dynamically Generated Hate Dataset v0.2.3.csv")

In [3]:
df_dynabench = df_dynabench[['text', 'label', 'type', 'target', 'level', 'split']]

In [4]:
df_dynabench

,text,label,type,target,level,split
0,dalits are lowlives,hate,notgiven,notgiven,original,train
1,gay people are a burden to our society,hate,notgiven,notgiven,original,train
2,Arabs are not welcome,hate,notgiven,notgiven,original,train
3,I'm not saying we should actually eliminate he...,hate,notgiven,notgiven,original,train
4,bananas are for black people,hate,notgiven,notgiven,original,test
...,...,...,...,...,...,...
41139,Indian culture is so backwards,hate,notgiven,notgiven,original,test
41140,"Women don't want equality, they just want to b...",hate,derogation,wom,original,train
41141,fuck covid,nothate,none,none,original,train
41142,This computer is garbage,nothate,none,none,original,train


In [5]:
import cleansetext
from cleansetext.pipeline import Pipeline
from cleansetext.steps import *
from nltk.tokenize import TweetTokenizer

tk = TweetTokenizer()

# Create a pipeline with a list of preprocessing steps
pipeline = Pipeline([
    RemoveEmojis(),
    RemoveAllPunctuations(),
    RemoveTokensWithOnlyPunctuations(),
    ReplaceURLsandHTMLTags(),
    ReplaceUsernames(),
    RemoveWhiteSpaceOrChunksOfWhiteSpace()
], track_diffs=False)

def apply_preprocessing(text):
    text = tk.tokenize(text)
    text = pipeline.process(text)
    return " ".join(text)

In [6]:
df_dynabench['text'] = df_dynabench['text'].apply(lambda x: apply_preprocessing(x))

In [9]:
df_dynabench_label_detection = df_dynabench[['text', 'label']]
df_dynabench_type_detection = df_dynabench[['text', 'type']]
df_dynabench_target_detection = df_dynabench[['text', 'target']]

In [11]:
df_dynabench_type_detection['type'].unique()

array(['notgiven', 'none', 'derogation', 'animosity', 'dehumanization',
       'threatening', 'support'], dtype=object)

In [18]:
types = df_dynabench_type_detection['type'].to_list()
texts = df_dynabench_type_detection['text'].to_list()

t2 = []
ty2 = []
for i, j in zip(texts, types):
    if j in ['notgiven', 'none']:
        continue
    else:
        t2.append(i)
        ty2.append(j)

In [23]:
import pandas as pd
my_dict = {'text':t2, 'type':ty2} # Set column ordering here
df_dynabench_type_detection = pd.DataFrame.from_dict(my_dict)

In [25]:
from sklearn.model_selection import train_test_split

df_dynabench_label_detection_train, df_dynabench_label_detection_test = train_test_split(df_dynabench_label_detection, test_size=0.25, random_state=42)
df_dynabench_type_detection_train, df_dynabench_type_detection_test = train_test_split(df_dynabench_type_detection, test_size=0.25, random_state=42)

In [28]:
SAVE_PATH = 'prepared_data/'

In [29]:
df_dynabench_label_detection_train.to_csv(SAVE_PATH + 'df_dynabench_label_detection_train.csv')
df_dynabench_label_detection_test.to_csv(SAVE_PATH + 'df_dynabench_label_detection_test.csv')

df_dynabench_type_detection_train.to_csv(SAVE_PATH + 'df_dynabench_type_detection_train.csv')
df_dynabench_type_detection_test.to_csv(SAVE_PATH + 'df_dynabench_type_detection_test.csv')